## SICP 习题 （2.65）解题总结：新的并集和交集操作

SICP 习题2.65要求我们使用2.63和2.64的函数构建union-set函数和intersection-set函数，同时要求复杂度为O(n)

这道题我自己做的时候花了很长时间，当时主要考虑的是如何同时递归遍历两棵树，一边遍历一边生成新的平衡树。

这个困扰了我很长时间，因为要同时递归遍历两棵不同的树是一件非常诡异的事情，到现在我都还没有一个清晰的思路。

后来去网上看了看别人的解题思路，发现大家用了一个非常简单的方法，就是把树转成列表,用之前的union-set函数和intersection-set函数进行操作，然后再把列表转成树。。。。。

清奇的解题思路，因为所有涉及的函数都是O(n)，所以累积使用几个函数复杂度还是O(n)....

那下面就开始堆代码：

In [28]:
(define (list->tree elements)
  (car (partial-tree elements (length elements))))

(define (is-tree? tree)
  (if (pair? tree)
      (if (= (length tree) 3)
	  #t
	  #f)
      #f))

(define (entry tree) 
  (if (is-tree? tree) 
       (car tree)
       tree))


(define (left-branch tree)
  (if (is-tree? tree)
      (cadr tree)
      '()))

(define (right-branch tree) 
  (if (is-tree? tree)
      (caddr tree)
      '()))

(define (make-tree entry left right)
  (list entry left right))


(define (partial-tree elts n)
  (if (= n 0)
      (cons '() elts)
      (let ((left-size (quotient (- n 1) 2)))
	(let ((left-result (partial-tree elts left-size)))
	  (let ((left-tree (car left-result))
		(non-left-elts (cdr left-result))
		(right-size (- n (+ left-size 1))))
	    (let ((this-entry (car non-left-elts))
		  (right-result (partial-tree (cdr non-left-elts)
					    right-size)))
	      (let ((right-tree (car right-result))
		    (remaining-elts (cdr right-result)))
		(cons (make-tree this-entry left-tree right-tree)
		      remaining-elts))))))))

(define (list->tree elements)
  (car (partial-tree elements (length elements))))



In [29]:
(define test-elements-1 '(1 3 4 5 6 7 9 10))
(define test-elements-2 '(1 4 5 6 8 9 10 11))

(define target-tree-1
        (car (partial-tree test-elements-1 (length test-elements))))
(define target-tree-2
        (car (partial-tree test-elements-2 (length test-elements))))

In [30]:
(define (tree->list-2 tree)
  (define (copy-to-list tree result-list)
    (if (null? tree)
	result-list
	(copy-to-list (left-branch tree)
		      (cons (entry tree)
			    (copy-to-list (right-branch tree)
					  result-list)))))
  (copy-to-list tree '()))

In [31]:
(define target-list-1 (tree->list-2 target-tree-1))
(define target-list-2 (tree->list-2 target-tree-2))

In [32]:
(define (intersection-set set1 set2)
  (if (or (null? set1) (null? set2))
      '()
      (let (( x1 (car set1)) (x2 (car set2)))
	(cond ((= x1 x2)
	       (cons x1
		     (intersection-set (cdr set1)
				      (cdr set2))))
	      ((< x1 x2)
	       (intersection-set (cdr set1) set2))
	      ((< x2 x1)
	       (intersection-set set1 (cdr set2)))))))

In [33]:
(define (union-set set1 set2)
  (if (null? set1)
       set2
       (if (null? set2)
	   set1
	   (let (( x1 (car set1)) (x2 (car set2)))
	     (cond ((= x1 x2)
		    (cons x1
			  (union-set (cdr set1)
				     (cdr set2))))
		   ((< x1 x2)
		    (cons x1 (union-set (cdr set1) set2)))
		   ((< x2 x1)
		    (cons x2 (union-set set1 (cdr set2)))))))))



In [34]:
target-list-1

(1 3 4 5 6 7 9 10)

In [35]:
target-list-2

(1 4 5 6 8 9 10 11)

In [36]:
(intersection-set target-list-1 target-list-2)

(1 4 5 6 9 10)

In [37]:
(union-set target-list-1 target-list-2)

(1 3 4 5 6 7 8 9 10 11)

In [38]:
(list->tree (intersection-set target-list-1 target-list-2))

(5 (1 () (4 () ())) (9 (6 () ()) (10 () ())))

In [39]:
(list->tree (union-set target-list-1 target-list-2))

(6 (3 (1 () ()) (4 () (5 () ()))) (9 (7 () (8 () ())) (10 () (11 () ()))))